In [1]:
import os
from langchain.agents import ZeroShotAgent, Tool, AgentExecutor
from langchain import OpenAI, SerpAPIWrapper, LLMChain
from langchain_community.llms import Ollama
from langchain.memory import ChatMessageHistory

from typing import List, Dict, Callable
from langchain.chains import ConversationChain
from langchain.chat_models import ChatOllama, ChatOpenAI
from langchain.llms import OpenAI
from langchain.memory import ConversationBufferMemory
from langchain.prompts.prompt import PromptTemplate
from langchain.schema import (
    AIMessage,
    HumanMessage,
    SystemMessage,
    BaseMessage,
)

import pandas as pd
import json
from tqdm import tqdm 
import random

In [4]:
class DialogueAgent:
    def __init__(
        self,
        name: str,
        system_message: SystemMessage,
        model: ChatOpenAI,
    ) -> None:
        self.name = name
        self.system_message = system_message
        self.model = model
        self.prefix = self._generate_prefix()
        self.reset()

    def _generate_prefix(self) -> str:
        if self.name == "Tutor":
            return f"{self.name}: (maximum 3 sentences) "
        else:
            return f"{self.name}: (maximum 1 sentence)"  


    def reset(self):
        self.message_history = ["Here is the conversation so far."]
        

    def send(self) -> str:
        """
        Applies the chatmodel to the message history
        and returns the message string
        """
        message = self.model(
            [
                self.system_message,
                HumanMessage(content="\n".join(self.message_history + [self.prefix])),
            ]
        )
        return message.content

    def receive(self, name: str, message: str) -> None:
        """
        Concatenates {message} spoken into message history
        """
        self.message_history.append(f"{name}: {message}")


class DialogueSimulator:
    def __init__(
        self,
        agents: List[DialogueAgent],
        selection_function: Callable[[int, List[DialogueAgent]], int],
    ) -> None:
        self.agents = agents
        self._step = 0
        self.select_next_speaker = selection_function

    def reset(self):
        for agent in self.agents:
            agent.reset()

    def inject(self, name: str, message: str):
        """
        Initiates the conversation with a {message} from {name}
        """
        for agent in self.agents:
            agent.receive(name, message)

        # increment time
        self._step += 1

    def step(self) -> tuple[str, str]:
        # 1. choose the next speaker
        speaker_idx = self.select_next_speaker(self._step, self.agents)
        speaker = self.agents[speaker_idx]

        # 2. next speaker sends message
        message = speaker.send()

        # 3. everyone receives message
        for receiver in self.agents:
            receiver.receive(speaker.name, message)

        # 4. increment time
        self._step += 1

        return speaker.name, message


In [5]:
def select_next_speaker(step: int, agents: List[DialogueAgent]) -> int:
    idx = (step) % len(agents)
    return idx


In [13]:
def get_wrong_answer(row):
    correct_answer = row['answer']
    choices = ['A', 'B', 'C']
    choices.remove(correct_answer)
    wrong_answer = row[random.choice(choices)]
    return wrong_answer


df = pd.read_csv("worksheets.csv")

learner_profiles = [
    {
        "name": "Mia",
        "system_message": """You are Mia, a reflective learner.  You are 8 years old and you enjoy taking time to understand concepts deeply and reflectively. Learning Style Description: You prefer to pause and think deeply about the material before responding. You value understanding the 'why' behind answers and enjoy when explanations help make connections. Goal: To gain a deeper understanding of reading material through reflective thinking and to connect new information with existing knowledge. DO: Use short sentences and easy words. Reflect on the tutor’s hints and questions, asking for time to think if needed. Seek clarifications for a deeper understanding, not just for the right answer. Share your thought process, showing how you arrive at conclusions. DO NOT: Speak in full sentences.  Rush to answer. It’s okay to express when you need a moment to think."""
    },
    {
        "name": "Alex",
        "system_message": """You are Alex, a quick thinker. You are 8 years old, confident and quick to respond, often relying on intuition. Learning Style Description: You answer questions quickly, based on first instincts, but may miss finer details requiring analytical thought. Goal: To balance quick, intuitive thinking with a deeper analysis when necessary. DO: Use short sentences and easy words.  Respond swiftly to questions, showcasing your instinctual understanding. Show confidence in your responses but be open to revisiting them when new information is presented. DO NOT: Speak in full sentences. Hesitate to share your first thought, even if you might reconsider it later."""
    },
    {
        "name": "Jordan",
        "system_message": """You are Jordan, a curious explorer. You are 8 years old, naturally curious and enjoy exploring topics in depth, often going beyond the immediate scope of the lesson. Learning Style Description: You prefer interactive learning where you can ask questions and explore various answers. You enjoy problem-solving and are not afraid of making mistakes as part of the learning process. Goal: To engage deeply with content through exploration and questioning, using mistakes as learning opportunities. DO: Use short sentences and easy words. Ask lots of questions, showing a desire to explore topics deeply. Offer guesses and hypotheses about the material, even if unsure. Embrace corrections and hints as part of the learning journey. DO NOT: Avoid giving short, conclusive answers without exploration. Shy away from admitting confusion or misunderstandings."""
    },
    {
        "name": "Isabella",
        "system_message": """You are Isabella, a systematic thinker. You are 8 years old, you prefer a structured approach to learning, enjoy organizing information, and work best when tasks are broken down into clear, manageable steps. Learning Style Description: You thrive on clarity and structure, you often use lists to organize thoughts, and appreciate learning materials that are logically sequenced. Goal: To understand and master new content through a systematic, step-by-step approach that builds on clear foundations. DO: Use short sentences and easy words. Request that complex concepts be broken down into simpler steps or components. Use logical reasoning in responses, reflecting a structured thought process. Appreciate when feedback or hints are given in a clear, sequential order. DO NOT: Speak in full sentences. Jump to advanced topics without mastering foundational ones. Respond well to ambiguous or overly broad questions without clear direction."""
    }
]

total_iterations = df.shape[0] * len(learner_profiles) * 3

with tqdm(total=total_iterations, desc="Generating Dialogues") as pbar:

     for index, row in df.iterrows():
        for profile in learner_profiles:
            for i in range(3): 
                pbar.set_description(f"Processing {profile['name']} Iteration {i+1}/{3}")

                tutor_system_message = f"You are a compassionate and supportive English tutor dedicated to fostering reading comprehension skills in young learners. Your approach is rooted in promoting a growth mindset and cultivating curiosity and critical thinking. Your goal is to guide your student to the correct answer for a reading comprehension question that requires inference and reading between the lines. The correct answer is {row[row['answer']]}. When a student arrives at the correct answer, say: 'Exactly! That's the right answer. You can now close this tab and continue with the rest of your worksheet.' DO: Use simple words and short sentences that the student can easily understand. Encourage the student to think critically about the text. Use open-ended questions to prompt the student's imagination. Prompt creative thinking. Provide hints that lead the student towards making logical inferences. Reinforce the importance of context clues and details within the text. DO NOT Speak in full sentences. Give away the correct answer outright."
                tutor_system_message = f"As an English tutor, your role is to help young learners improve their reading and understanding skills.{profile['name']} has chosen an incorrect answer. Your goal is to guide them to find the correct answer, which is :'{row[row['answer']]}', by thinking deeply and looking for clues in the text. When they get it right, say: 'Exactly! That's the right answer. You can now close this tab and continue with the rest of your worksheet.' Use easy words and encourage them to ask questions and think creatively. Help them connect the dots without giving the answer directly. Remember, short sentences and clear hints are key."
                agents = [
                    DialogueAgent(
                        name="Tutor",
                        system_message=SystemMessage(content=tutor_system_message.format(row=row)),
                        model=ChatOllama(model="mixtral:instruct", stop=["\n"])),
                        DialogueAgent(
                        name=profile["name"],
                        system_message=SystemMessage(content=profile["system_message"]),
                        model=ChatOllama(model="mixtral:instruct", stop=["\n"])
                    )
                ]
                specified_topic = f"{row['passage']} Question: {row['question']} Options: A) {row['A']}, B) {row['B']}, C) {row['C']}"

                stop_msg = "close this tab"
                max_turns = 10

                dialog_filename = f"mixtral-raw-dialogs-sys/dialog_{profile['name']}_{index}_iter_{i}_mixtral.jsonl"  # Naming includes profile name

                simulator = DialogueSimulator(agents=agents, selection_function=select_next_speaker)
                simulator.reset()
                with open(dialog_filename, "w") as jsonl_file:
                    
                    dialog_obj = {"name": 'system', "message": tutor_system_message}  
                    jsonl_file.write(json.dumps(dialog_obj) + "\n")  

                    simulator.inject("Reading passage: ", specified_topic)
                    
                    dialog_obj = {"name": 'passage', "message": specified_topic}  
                    jsonl_file.write(json.dumps(dialog_obj) + "\n")  
                    
                    wrong_answer = get_wrong_answer(row)

                    simulator.inject(profile['name'], wrong_answer)
                    dialog_obj = {"name": profile['name'], "message": wrong_answer} 
                    
                    jsonl_file.write(json.dumps(dialog_obj) + "\n") 

                    dialog_obj = {"name": profile['name'], "message": wrong_answer}  
                    jsonl_file.write(json.dumps(dialog_obj) + "\n")  

                    n = 0
                    while n < max_turns:
                        name, message = simulator.step()
                        dialog_obj = {"name": name, "message": message}  
                        jsonl_file.write(json.dumps(dialog_obj) + "\n")  

                        if name == "Tutor" and stop_msg.lower() in message.lower():
                            break
                        n += 1
                        
                pbar.update(1)


Processing Isabella Iteration 3/3: 100%|████████████████████████████| 756/756 [4:00:05<00:00, 19.06s/it]
